## define functions

- contains requered number -> ensure only specific images are pull
- get image -> pull the require and compress image
- scrapp_image_description -> iterate for each of the products
- grucci -> iterate scrap_image

### scrapper

In [3]:
import requests
from PIL import Image
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from io import BytesIO
#import pandas as pd
from openpyxl import load_workbook
#import psycopg2


In [5]:
from DB_and_Azure import sql_db_functions as SQLf, azure_functions as Af

In [4]:
def create_soup(url):

    ua = UserAgent()
    header = {'User-Agent':str(ua.chrome)}

    # Send an HTTP request to the URL
    response = requests.get(url, headers = header)
    soup = BeautifulSoup(response.content, 'html.parser')

    return soup


In [8]:
def contains_required_number(line):
    
    keeping = ['001_100','004_100','005_100']
    # 001: Prod
    # 004: back
    # 005: 3/4 image
    return any(num in line for num in keeping)

In [9]:
def get_image_links(prod_soup):
    
    item_list_1 = prod_soup.find_all(attrs={"data-image-size": "small-retina"})

    ##### get image
    images = []
    i = 0
    for item in item_list_1: 
            
        i+=1 
        if (i == len(item_list_1)) | (i == 1):
            continue
        elif not(contains_required_number(item['srcset'])) :
            continue
        else:
            images = images + [item['srcset']]
        
        i += 1
    
    return images

In [10]:
def get_description(prod_soup):
    
    description_list = prod_soup.find_all(attrs={"class": "product-detail"} )

    #Get description
    description = ''
    
    for a in description_list[0].ul.find_all('li'):
        description = description + ' ' + a.text
        

    return description_list[0].p.text.replace('\n', '').replace('\t', '') + ' ' + description

    

In [11]:
def links_to_images(links,prod_name,container_client):

    ua = UserAgent()
    header = {'User-Agent':str(ua.chrome)}

    i = 0
    for img_link in links:
        response = requests.get('https:' + img_link, headers = header)
        img = Image.open(BytesIO(response.content))

        # Resize the image to 600
        img_resized = img.resize((500, 500))

        img_name = f"Gucci{prod_name}_{i}.jpg"

        # Load to Blob
        Af.images_to_blob(img, img_name,container_client)
        i+=1



In [15]:
def gucci(category_url,n_products):

    soup = create_soup(category_url)
    item_list = soup.select('article a')

    
    container_client = Af.connect_blob()

    conn, cursor = SQLf.connect_sql()


    br = 0
    for item in item_list: 
            
        if br == n_products:
            break
        else:

            #Scrapp
            prod_soup = create_soup('https://www.gucci.com' + item['href'])

            prod_images_links = get_image_links(prod_soup=prod_soup)
            prod_description = get_description(prod_soup=prod_soup)
            #Get Price

            prod_name = 'HM_'+str(br)  
            
            # Links to image, load to blob and return prod_images_names
            prod_images_names = links_to_images(links = prod_images_links,prod_name= br,container_client=container_client)
            
            SQLf.insert_description_image_to_db(conn = conn, brand='HM', descript = prod_description, price = 10,image_names = prod_images_names)

            #data_to_SQL(prod_name=prod_name, prod_description=prod_description,brand='HM',prod_images_names=prod_images_names,cursor=cursor,conn=conn)

        br += 1


## Execute Scrapper

In [13]:
# Define the URL you want to scrape
url = 'https://www.gucci.com/it/it/ca/women/ready-to-wear-for-women-c-women-readytowear'

gucci(url,3)

Error: could not translate host name "('localhost'" to address: No such host is known. 
could not translate host name ")" to address: No such host is known. 

HM_0
Error: could not translate host name "('localhost'" to address: No such host is known. 
could not translate host name ")" to address: No such host is known. 

HM_1
Error: could not translate host name "('localhost'" to address: No such host is known. 
could not translate host name ")" to address: No such host is known. 

HM_2
